In [1]:
import os
%pwd

'f:\\ml_projects\\text_summarizer\\notbooks'

In [2]:
os.chdir('../')
%pwd

'f:\\ml_projects\\text_summarizer'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(fronzer=True)
class DataTransformationConfig:
    